# Elevated-plus maze analysis

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats
import os
import itertools

In [ ]:
def cdf(array, ax=None, ignore_nan=True, **kwargs):
    '''Creates cumulative distribution function from array of data points
    Parameters
    ----------
    array : array_like
        Array to plot CDF. If more than one dimension, array will be flattened.
    ax : Axes object, optional
        Axes on which to plot CDF.
    ignore_nan : boolean
        Whether to ignore nan values in `array`.
    **kwargs: keyword arguments
        Keyword arguments for matplotlib.pyplot.plot.

    Returns
    ----------
    ax: axes object containing CDF

    '''
    array = np.array(array).flatten()
    if ignore_nan:
        array = array[~ np.isnan(array)]

    X = np.sort(array)
    Y = (np.arange(len(array), dtype=float) + 1) / len(array)
    if ax:
        ax.plot(X, Y, **kwargs)
    else:
        plt.plot(X, Y, **kwargs)

    return ax

In [ ]:
neural_file = 'epm-neural.csv'
behav_file = 'epm-behav.csv'

save_plot = False
output_dir = './results'
if output_dir and not os.path.isdir(output_dir):
    os.mkdir(output_dir)

idx = pd.IndexSlice

# Import and set up data

In [ ]:
neural = pd.read_csv(neural_file, header=[0, 1], index_col=0)
behav = pd.read_csv(behav_file, header=[0, 1], index_col=0)

## Bin neural activity
Calculate mean neural activity in discrete bins along arms of EPM.

In [ ]:
n_bins = 10
max_d = 35
bins = np.arange(n_bins + 1, dtype=float) / n_bins * max_d
bins = np.insert(bins, 0, -1)
bin_labels = np.arange(0, n_bins+1) / 10.
binned_loc = pd.cut(behav.xs('distance_from_center', axis=1, level='feature').stack('subject'), bins, labels=bin_labels).unstack('subject')
binned_loc.columns = pd.MultiIndex.from_product([binned_loc.columns, ['distance_from_center']], names=['subject', 'feature'])

df_zone = behav.loc[:, (slice(None), ['in_center', 'in_closed', 'in_open'])].astype(bool).sort_index(axis=1)
for subj, df in df_zone.groupby(axis=1, level='subject'):
    df_zone[(subj, 'zone')] = pd.Series(['center' if x else 'closed' if y else 'open' if z else 'na' for (x, y, z), in zip(df.values)], index=df.index)

df_loc = pd.concat([binned_loc, df_zone], axis=1).sort_index(axis=1)

loc = df_loc.loc[:, (slice(None), ['zone', 'distance_from_center'])]
dfs = {}
for subj, df in df_loc.groupby(axis=1, level='subject'):
    dfs[subj] = pd.concat([loc[subj], neural[subj]], axis=1).groupby(['distance_from_center', 'zone']).mean()

neural_zone = pd.concat(dfs, axis=1, names=['subject', 'neuron'])

## Group neurons
Determine response of neurons based on difference in activity between open and closed arm using Mann Whitney U test.

In [ ]:
# Average activity by location

n_cells = neural.shape[1]
p_thresh = 0.05 / n_cells #neural_arm.shape[0]
center_col = 'in_center'
closed_col = 'in_closed'
open_col = 'in_open'

# Average activity for each neuron in each zone
neural_arm = pd.DataFrame(
    index=neural.columns,
    columns=['center', 'open', 'closed'],
    dtype=float
)
neural_arm.columns.name = 'arm'
neural_response = pd.DataFrame(index=['group', 'difference', 'p'], columns=neural.columns, dtype=float)

for subj, df in neural.groupby(axis=1, level='subject'):
    center_ix = behav[(subj, center_col)] == 1
    closed_ix = behav[(subj, closed_col)] == 1
    open_ix = behav[(subj, open_col)] == 1
    
    neural_arm.loc[subj, 'center'] = df.loc[center_ix, :].mean(axis=0)
    neural_arm.loc[subj, 'closed'] = df.loc[closed_ix, :].mean(axis=0)
    neural_arm.loc[subj, 'open'] = df.loc[open_ix, :].mean(axis=0)
    
    for cell in df.loc[:, subj]:
        X1 = df.loc[closed_ix, (slice(None), cell)]
        X2 = df.loc[open_ix, (slice(None), cell)]
        _, neural_response.loc['p', (subj, cell)] = scipy.stats.mannwhitneyu(X1, X2)

# Calculate difference between arms
neural_response.loc['difference'] = (neural_arm['open'] - neural_arm['closed'])
significant_ix = neural_response.loc['p'] < p_thresh

# Define groups
neural_response.loc['group'] = 0
exc_ix = neural_response.loc['difference'] > 0
inh_ix = neural_response.loc['difference'] < 0
neural_response.loc['group', neural_response.columns[significant_ix & exc_ix]] = 1
neural_response.loc['group', neural_response.columns[significant_ix & inh_ix]] = -1

# Add groups to `neural_arm`
neural_arm.index = pd.MultiIndex.from_tuples(
    [x + (neural_response.loc['group', x], ) for x in neural_arm.index],
    names = ['subject', 'neuron', 'response']
)

In [ ]:
groups = neural_response.loc['group', :].astype(int).to_dict()
neural_grp = neural.copy()
neural_grp.columns = pd.MultiIndex.from_tuples([x + (groups[x], ) for x in neural], names=neural.columns.names + ['group'])

## Neural-behavior Correlations
Calculate Spearman correlation coefficient between behavioral features and neural activity within the open arm.

In [ ]:
# Calculate distance-activity correlations

features = ['distance_from_center', 'velocity']
neural_corr = pd.DataFrame(
    index=neural_grp.columns,
    columns=pd.MultiIndex.from_product(
        [['open', 'closed'], features, ['r', 'p']],
        names=['arm', 'feature', 'stat']
    ),
    dtype=float
)
neural_corr_stats = pd.DataFrame(
    index=[-1, 0, 1],
    columns=pd.MultiIndex.from_product(
        [['open', 'closed'], features, ['t', 'p']],
        names=['arm', 'feature', 'stat']
    )
)

for feature in features:
    for subj, df_subj in neural_grp.groupby(axis=1, level='subject'):
        closed_ts = behav.index[behav[subj, 'in_closed'] == 1]
        open_ts = behav.index[behav[subj, 'in_open'] == 1]
        for cell in df_subj:
            for arm, arm_ix in zip(['open', 'closed'], [open_ts, closed_ts]):
                X = df_subj.loc[arm_ix, cell]
                Y = behav[subj, feature][arm_ix]
                ix = ~(X.isna() | Y.isna())

                (
                    neural_corr.loc[cell, (arm, feature, 'r')],
                    neural_corr.loc[cell, (arm, feature, 'p')]
                ) = scipy.stats.spearmanr(X[ix], Y[ix])

# Stats on correlation distributions
for arm in ['closed', 'open']:
    for feature in features:
        for grp in [-1, 0, 1]:
            neural_corr_stats.loc[grp, (arm, feature)] = list(scipy.stats.ttest_1samp(
                np.arctanh(neural_corr.loc[idx[:, :, grp], (arm, feature, 'r')]),
                0,
                nan_policy='omit'
            ))

# Visualization

### Figure 4D

In [ ]:
fig, axes = plt.subplots(ncols=2)

cdf(neural_arm['closed'].values, ax=axes[0], label='closed')
cdf(neural_arm['open'].values, ax=axes[0], label='open')
axes[0].legend()
axes[0].set_xlabel('mean_response')

data = neural_arm[['closed', 'open']]
ax0 = sns.stripplot(ax=axes[1], data=data.melt(), x='arm', y='value', s=3, jitter=True, zorder=0)
ax1 = sns.pointplot(ax=axes[1], data=data.melt(), x='arm', y='value', scale=0.5, color='k', ci=68, join=False)
# axes[1].errorbar(range(2), data.mean(), data.sem(), c='k', fmt='_', zorder=100)

test_val, p_val = scipy.stats.wilcoxon(neural_arm['open'], neural_arm['closed'])
fig.suptitle('Wilcoxon signed-rank test\nW: {}\np: {}'.format(test_val, p_val))
fig.tight_layout(rect=[0, 0.03, 1, 0.85])

if save_plot:
    plt_name = os.path.join(output_dir, 'fig4d')
    fig.savefig(plt_name + '.pdf')
    fig.savefig(plt_name + '.png')

### Figure 4E

In [ ]:
counts = neural_arm.groupby('response').count()['center']

fig, ax = plt.subplots()
counts.plot.pie(ax=ax, autopct='%1.1f%%', wedgeprops={'lw': 5, 'ec': 'w'})
ax.axis('image')
centre_circle = plt.Circle((0, 0), 1 / 1.4142, color='none', fc='white')
ax.add_artist(centre_circle);

if save_plot:
    plt_name = os.path.join(output_dir, 'fig4e')
    fig.savefig(plt_name + '.pdf')
    fig.savefig(plt_name + '.png')

### Figure 4H,I

In [ ]:
data = neural_zone.copy()
data = data.drop(itertools.product(bin_labels[1:], ['center']))
data = data.drop(itertools.product([0.0], ['closed', 'open']))
data = data.drop('na', level='zone')

n_rows = data.shape[0]
data = data.iloc[list(range(n_rows-2, 0, -2)) + list(range(0, n_rows, 2))]

sort_ix_peak = np.nanargmax(data.values, axis=0).argsort()
sort_ix = data.mean(axis=0, level='zone')
sort_ix = sort_ix.loc['open'] / sort_ix.loc['closed']
sort_ix_mean = sort_ix.values.argsort()

gridspec_kw = {'height_ratios': [3, 1], 'width_ratios': [2, 0.1]}
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(6, 8), sharex='col', gridspec_kw=gridspec_kw)
sns.heatmap(data.iloc[:, sort_ix_mean].T, ax=axes[0, 0], center=0, vmax=2, cbar_ax=axes[0, 1]);

data_long = data.copy()
data_long.columns = pd.MultiIndex.from_tuples(
    [x + (y, ) for x, y in zip(data.columns, neural_response.loc['group', data.columns].astype(int))],
    names=data.columns.names + ['group']
)
data_long = data_long.stack(list(range(data_long.columns.nlevels))).reset_index().rename(columns={0: 'neural'})
data_long['arm_loc'] = data_long['zone'] + ' ' + data_long['distance_from_center'].astype(str)
zone_order = (
    [' '.join(x) for x in itertools.product(['closed'], bin_labels[-1:0:-1].astype(str))] +
    ['center 0.0'] +
    [' '.join(x) for x in itertools.product(['open'], bin_labels[1:].astype(str))]
)
sns.lineplot(data=data_long, x='arm_loc', y='neural', hue='group',
             hue_order=[-1, 0, 1], ci=68, ax=axes[1, 0], sort=False)
for ax in axes[1]:
    plt.setp(ax.get_xticklabels(), rotation=90)
sns.despine(ax=axes[1, 0])
axes[1, 1].axis('off')

fig.tight_layout()

if save_plot:
    plt_name = os.path.join(output_dir, 'fig4hi')
    fig.savefig(plt_name + '.pdf')
    fig.savefig(plt_name + '.png')

### Figure 4J,K

In [ ]:
neural_corr_long = neural_corr['open'].xs('r', axis=1, level='stat').stack(list(range(neural_corr.columns.nlevels - 2))).reset_index().rename(columns={0: 'r'})
features = ['distance_from_center', 'velocity']
g = sns.FacetGrid(
    neural_corr_long, row='feature', hue='group',
    row_order=features, hue_order=[-1, 0, 1], aspect=1.25
)
g.map(cdf, 'r').add_legend()

# Format
for ax, feature in zip(g.axes.flatten(), features):
    grp_stats = neural_corr_stats[('open', feature)]
    ax.set_xlim(-0.75, 0.75)
    ax.set_title(
        f'{ax.get_title()}\n'
        f"inh | W: {grp_stats.loc[-1, 't']:.2e} p: {grp_stats.loc[-1, 'p']:.2e}\n"
        f"nc | W: {grp_stats.loc[0, 't']:.2e} p: {grp_stats.loc[0, 'p']:.2e}\n"
        f"exc | W: {grp_stats.loc[1, 't']:.2e} p: {grp_stats.loc[1, 'p']:.2e}\n"
    )
g.fig.tight_layout(rect=[0, 0, 0.95, 1])

if save_plot:
    plt_name = os.path.join(output_dir, 'fig4jk')
    g.fig.savefig(plt_name + '.pdf')
    g.fig.savefig(plt_name + '.png')